In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
import sklearn
import numpy as np
import pandas as pd
import gc

In [47]:

# read in the csv containing the urls and type
df = pd.read_csv('/content/drive/Shareddrives/Machine Learning Final Project/data/malicious_urls.csv')
df['type'].unique()

array(['phishing', 'benign', 'defacement', 'malware'], dtype=object)

In [48]:
# encode benign to be 0 and all the malicious types to be 1
mal_type = {'benign': 0, 'defacement': 1, 'phishing': 2, 'malware': 3}
df['label'] = [mal_type[item] for item in df['type']]
df

,url,type,label
0,br-icloud.com.br,phishing,2
1,mp3raid.com/music/krizz_kaliko.html,benign,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1
...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2


In [49]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(df['type'])
test = encoder.transform(df['type'])
hot = np_utils.to_categorical(test)

In [50]:
df['url_length'] = [len(item) for item in df['url']]
df

,url,type,label,url_length
0,br-icloud.com.br,phishing,2,16
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235
...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45


In [51]:
df['backslash_count'] = [item.count('/') for item in df['url']]
df

,url,type,label,url_length,backslash_count
0,br-icloud.com.br,phishing,2,16,0
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3
...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2


In [52]:
# encode none to be 0, http to be 1, and https to be 2
df['http_type'] = [2 if ('https://') in item else (1 if ('http://') in item else 0) for item in df['url']]
df

,url,type,label,url_length,backslash_count,http_type
0,br-icloud.com.br,phishing,2,16,0,0
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1
...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0


In [53]:
!pip install tldextract
import tldextract
url_info = [tldextract.extract(item) for item in df['url']]
df['subdomain'] = [item.subdomain if len(item.subdomain) > 0 else None for item in url_info]
df['suffix'] = [item.suffix if len(item.suffix) > 0 else None for item in url_info]
del url_info
df

,url,type,label,url_length,backslash_count,http_type,subdomain,suffix
0,br-icloud.com.br,phishing,2,16,0,0,None,com.br
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0,None,com
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0,None,org
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1,www,be
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1,None,net
...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0,xbox360,com
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0,games,com
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0,www,com
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0,en,org


In [54]:
df['periods_in_suffix'] = [item.count('.') if not item == None else (0) for item in df['suffix']]
df

,url,type,label,url_length,backslash_count,http_type,subdomain,suffix,periods_in_suffix
0,br-icloud.com.br,phishing,2,16,0,0,None,com.br,1
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0,None,com,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0,None,org,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1,www,be,0
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1,None,net,0
...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0,xbox360,com,0
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0,games,com,0
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0,www,com,0
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0,en,org,0


In [55]:
df['trimmed_suffix'] = [item.split('.')[0] if not item == None else (None) for item in df['suffix']]
df

,url,type,label,url_length,backslash_count,http_type,subdomain,suffix,periods_in_suffix,trimmed_suffix
0,br-icloud.com.br,phishing,2,16,0,0,None,com.br,1,com
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0,None,com,0,com
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0,None,org,0,org
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1,www,be,0,be
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1,None,net,0,net
...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0,xbox360,com,0,com
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0,games,com,0,com
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0,www,com,0,com
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0,en,org,0,org


In [56]:
df['contains_percent'] = [1 if ('%') in item else (0) for item in df['url']]
df

,url,type,label,url_length,backslash_count,http_type,subdomain,suffix,periods_in_suffix,trimmed_suffix,contains_percent
0,br-icloud.com.br,phishing,2,16,0,0,None,com.br,1,com,0
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0,None,com,0,com,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0,None,org,0,org,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1,www,be,0,be,0
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1,None,net,0,net,0
...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0,xbox360,com,0,com,0
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0,games,com,0,com,0
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0,www,com,0,com,0
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0,en,org,0,org,0


In [57]:

from urllib.parse import urlparse
from os.path import splitext

def get_ext(url):
    """Return the filename extension from url, or ''."""
    parsed = urlparse(url)
    root, ext = splitext(parsed.path)
    return ext[1:]  # or ext[1:] if you don't want the leading '.'


In [58]:
df['file_extension'] = [get_ext(item) if len(get_ext(item)) > 0 else None for item in df['url']]
df

,url,type,label,url_length,backslash_count,http_type,subdomain,suffix,periods_in_suffix,trimmed_suffix,contains_percent,file_extension
0,br-icloud.com.br,phishing,2,16,0,0,None,com.br,1,com,0,br
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0,None,com,0,com,0,html
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0,None,org,0,org,0,htm
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1,www,be,0,be,0,php
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1,None,net,0,net,0,php
...,...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0,xbox360,com,0,com,0,html
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0,games,com,0,com,0,None
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0,www,com,0,com,0,None
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0,en,org,0,org,0,None


In [59]:
from collections import defaultdict

# encode all the various domain suffixes with a default value of 0
suffix = defaultdict(lambda: 0)
suffix['com'] = 1
suffix['org'] = 2
suffix['net'] = 3
suffix['gov'] = 4
suffix['edu'] = 5

df['trimmed_suffix'] = [suffix[item] for item in df['trimmed_suffix']]
df

,url,type,label,url_length,backslash_count,http_type,subdomain,suffix,periods_in_suffix,trimmed_suffix,contains_percent,file_extension
0,br-icloud.com.br,phishing,2,16,0,0,None,com.br,1,1,0,br
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0,None,com,0,1,0,html
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0,None,org,0,2,0,htm
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1,www,be,0,0,0,php
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1,None,net,0,3,0,php
...,...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0,xbox360,com,0,1,0,html
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0,games,com,0,1,0,None
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0,www,com,0,1,0,None
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0,en,org,0,2,0,None


In [60]:
# encode all the various domain suffixes with a default value of 0
subdomain = defaultdict(lambda: 0)
subdomain['www'] = 1

df['subdomain'] = [subdomain[item] for item in df['subdomain']]
df

,url,type,label,url_length,backslash_count,http_type,subdomain,suffix,periods_in_suffix,trimmed_suffix,contains_percent,file_extension
0,br-icloud.com.br,phishing,2,16,0,0,0,com.br,1,1,0,br
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0,0,com,0,1,0,html
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0,0,org,0,2,0,htm
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1,1,be,0,0,0,php
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1,0,net,0,3,0,php
...,...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0,0,com,0,1,0,html
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0,0,com,0,1,0,None
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0,1,com,0,1,0,None
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0,0,org,0,2,0,None


In [61]:
# encode the various file extensions with a default value of 0
file_ext = defaultdict(lambda: 0)
file_ext['html'] = 1
file_ext['php'] = 2
file_ext['aspx'] = 3

df['file_extension'] = [file_ext[item] for item in df['file_extension']]
df

,url,type,label,url_length,backslash_count,http_type,subdomain,suffix,periods_in_suffix,trimmed_suffix,contains_percent,file_extension
0,br-icloud.com.br,phishing,2,16,0,0,0,com.br,1,1,0,0
1,mp3raid.com/music/krizz_kaliko.html,benign,0,35,2,0,0,com,0,1,0,1
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,31,3,0,0,org,0,2,0,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1,88,3,1,1,be,0,0,0,2
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1,235,3,1,0,net,0,3,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,2,39,3,0,0,com,0,1,0,1
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,2,44,4,0,0,com,0,1,0,0
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,2,42,4,0,1,com,0,1,0,0
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,2,45,2,0,0,org,0,2,0,0


In [62]:
y = df['label']
df.drop(['type', 'label', 'url', 'suffix'], axis=1, inplace=True)
df

,url_length,backslash_count,http_type,subdomain,periods_in_suffix,trimmed_suffix,contains_percent,file_extension
0,16,0,0,0,1,1,0,0
1,35,2,0,0,0,1,0,1
2,31,3,0,0,0,2,0,0
3,88,3,1,1,0,0,0,2
4,235,3,1,0,0,3,0,2
...,...,...,...,...,...,...,...,...
651186,39,3,0,0,0,1,0,1
651187,44,4,0,0,0,1,0,0
651188,42,4,0,1,0,1,0,0
651189,45,2,0,0,0,2,0,0


In [63]:
#some model
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split

normalized_df=(df-df.min())/(df.max()-df.min())

X_train, X_test, y_train, y_test = train_test_split(normalized_df, hot, test_size=0.33, random_state=100)

In [64]:
#Model with 5 layers
model = Sequential()
model.add(Dense(90, input_shape = (None,X_train.shape[1]), activation = 'softmax'))
model.add(Dense(50, activation='relu', kernel_initializer="uniform"))
model.add(Dense(24, activation='relu', kernel_initializer="uniform"))
model.add(Dense(12, activation='relu', kernel_initializer="uniform"))
model.add(Dense(4, activation='sigmoid', kernel_initializer='uniform'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics= ['accuracy'])

In [ ]:
#fit our data to model
history = model.fit(X_train, y_train, validation_split = 0.33, epochs = 250, batch_size = 5000)

Epoch 1/250
59/59 [==============================] - 5s 14ms/step - loss: 1.3534 - accuracy: 0.6499 - val_loss: 1.2990 - val_accuracy: 0.6561
Epoch 2/250
59/59 [==============================] - 1s 9ms/step - loss: 1.1534 - accuracy: 0.6583 - val_loss: 1.0144 - val_accuracy: 0.6561
Epoch 3/250
59/59 [==============================] - 1s 10ms/step - loss: 0.9944 - accuracy: 0.6583 - val_loss: 0.9855 - val_accuracy: 0.6561
Epoch 4/250
59/59 [==============================] - 1s 9ms/step - loss: 0.9642 - accuracy: 0.6583 - val_loss: 0.9406 - val_accuracy: 0.6561
Epoch 5/250
59/59 [==============================] - 1s 9ms/step - loss: 0.8690 - accuracy: 0.6583 - val_loss: 0.7880 - val_accuracy: 0.6561
Epoch 6/250
59/59 [==============================] - 1s 10ms/step - loss: 0.7222 - accuracy: 0.7246 - val_loss: 0.6657 - val_accuracy: 0.7549
Epoch 7/250
59/59 [==============================] - 1s 9ms/step - loss: 0.6387 - accuracy: 0.7569 - val_loss: 0.6217 - val_accuracy: 0.7581
Epoch 8/25

In [ ]:
#test/predict 
from sklearn.metrics import confusion_matrix

label_predict = model.predict(X_test)

rounded = (label_predict  > 0.5).astype(int)

cm = confusion_matrix(y_test, rounded)

scores = model.evaluate(X_test,y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rounded, target_names=['Benign', 'Defacement', 'Phishing', 'Malware']))

In [ ]:
def print_cl_scores(scores):
  print('Accuracy: ' + str(np.mean(scores['test_accuracy'])))
  print('Precision: ' + str(np.mean(scores['test_precision'])))
  print('Recall: ' + str(np.mean(scores['test_recall'])))
  print('F1-Score: ' + str(np.mean(scores['test_f1'])))

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

X_train, X_holdout, y_train, y_holdout = train_test_split(df.values, y, test_size=0.3,random_state=17)

# for kNN, we need to scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, KFold

cl = RandomForestClassifier(criterion='gini', max_depth=10)
scores = cross_validate(cl, df, y, cv=KFold(n_splits=5, shuffle=True), scoring=["accuracy", "precision", "recall", "f1"])
print_cl_scores(scores)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, KFold

cl = KNeighborsClassifier(n_neighbors=5)
scores = cross_validate(cl, df, y, cv=KFold(n_splits=5, shuffle=True), scoring=["accuracy", "precision", "recall", "f1"])
print_cl_scores(scores)

In [ ]:
from sklearn.naive_bayes import GaussianNB
cl = GaussianNB()
scores = cross_validate(cl, df, y, cv=KFold(n_splits=5, shuffle=True), scoring=["accuracy", "precision", "recall", "f1"])
print_cl_scores(scores)

In [ ]:
from sklearn.linear_model import LogisticRegression
cl = LogisticRegression()
scores = cross_validate(cl, df, y, cv=KFold(n_splits=5, shuffle=True), scoring=["accuracy", "precision", "recall", "f1"])
print_cl_scores(scores)

In [ ]:
print(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
